In [1]:
import requests
import copy
import csv
import time
import json
import uuid
import pprint
import asyncio
from typing import List
from urllib import parse
from urllib.parse import quote
import inspect
import datetime

In [2]:
class magic_fstring_function:
    def __init__(self, payload):
        self.payload = payload
        self.cached = None
    def __str__(self):
        if self.cached is None:
            vars = inspect.currentframe().f_back.f_globals.copy()
            vars.update(inspect.currentframe().f_back.f_locals)
            self.cached = self.payload.format(**vars)
        return self.cached

In [3]:
uuid = str(uuid.uuid4())
item_list_api = "https://apis.zigbang.com/v2/items"
describe_list_api = item_list_api + '/list'
item_describe_api = "https://apis.zigbang.com/v3/items?item_ids={item_id}&detail=true"
item_view_url = "https://zigbang.com/home/oneroom/items/{item_id}"

referer = "https://zigbang.com/home/oneroom/subways/414/items"
        
headers = {
    'Host':'apis.zigbang.com',
    'Connection':'keep-alive',
    'Pragma':'no-cache',
    'Cache-Control':'no-cache',
    'Accept':'application/json, text/plain, */*',
    'Origin':'https://zigbang.com',
    'User-Agent':'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Mobile Safari/537.36',
    'DNT':'1',
    'Sec-Fetch-Site':'same-site',
    'Sec-Fetch-Mode':'cors',
    'Referer':'https://zigbang.com/home/oneroom/subways/414/items',
    'Accept-Encoding':'gzip, deflate, br',
    'Accept-Language':'ko-KR,ko;q=0.9,en-US;q=0.8,en;q=0.7',
}

items = {}
sad_words = [ '대출x', '대출불가', '전세 안됩']

In [4]:
def export_items(items):
    fieldnames = [
        'id',
        'title',
        'description',
        'deposit',
        'agent_name',
        'agent_mobile',
        '_floor',
        'address1',
        'address2',
        'address3',
        'size',
        'size_m2',
        'status',
        'room_type',
        'manage_cost_inc',
        'updated_at2',
        'url',
    ]
    
    filtered_item = [] 
    maximum_updated = datetime.timedelta(days=4)
    
    wish = '대출'
    for item in items:
        item = item['item']
        if wish in item['title'] or wish in item['description']:
            plz_flag = True
            for word in sad_words:
                if word in item['description']:
                    plz_flag = False
                    break
            if not plz_flag:
                continue
            item_updated = datetime.datetime.strptime(item['updated_at2'], '%Y-%m-%d')
            if maximum_updated < datetime.datetime.now() - item_updated:
                continue
            item_id = item['id']
            url = magic_fstring_function(item_view_url)
            item['url'] = str(url)
            new_item = {}
            for fieldname in fieldnames:
                new_item[fieldname] = item[fieldname]
            filtered_item.append(new_item)
    return filtered_item

In [5]:
def describe_room_list(items):
    item_ids = list(items.keys())
    max_idx = int(len(item_ids) / 30)
    if len(item_ids) % 30 != 0:
        max_idx += 1
    
    items = []
    headers['Accept'] = 'application/json'
    headers['Referer'] = 'https://zigbang.com/home/oneroom/subways/37/items'
    
    for i in range(max_idx):
        start_idx = i * 30
        end_idx = min((i+1)*30, len(item_ids))
        sub_ids = item_ids[start_idx:end_idx-1]
        item_id = str(sub_ids).replace(' ', '').replace('\'','')
        api = magic_fstring_function(item_describe_api)
        g = api
        resp = requests.get(
            url=str(api),
            headers=headers,
            timeout=5,
        )
        resp.encoding = 'utf-8'
        result = resp.json()
        resp.close()
        items += result['items']
        time.sleep(1)
        
    return items

In [6]:
def get_room_list(
    items,
    deposit_gteq: int = 0,
    deposit_lteq: int = 8000,
    domain: str = "zigbang",
    floor_in: str = "ground",
    geohash: str = "wydjr",
    rent_gteq: int = 0,
    sales_type_in: str = "전세",
    service_type_eq: str = "원룸",
):
    resp = requests.get(
        url=item_list_api,
        params={
            "deposit_gteq": deposit_gteq,
            "deposit_lteq": deposit_lteq,
            "domain": domain,
            "floor_in": floor_in,
            "geohash": geohash,
            "rent_gteq": rent_gteq,
            "sales_type_in": sales_type_in,
            "service_type_eq": service_type_eq,
        },
        headers=headers,
        timeout=5,
    )
    new_items = json.loads(resp.content.decode('utf-8'))
    resp.close()

    for section in new_items['sections']:
        for item in section['item_ids']:
            items[str(item)] = {}

    return items

In [7]:
items = {}


In [8]:
geos_sinrim = [
    'wydm0',
    'wydm1',
    'wydm2',
   'wydm3',
] 

for geo in geos_sinrim:
    items = get_room_list(items, geohash=geo)
    time.sleep(1)

In [9]:
len(items)

169

In [10]:
result = describe_room_list(items)

In [11]:
len(result)

163

In [12]:
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(result[0]['item'])

{ '_floor': '5층',
  'address1': '서울시 관악구 봉천동',
  'address2': '1667-21',
  'address3': None,
  'agent_address1': '서울특별시 관악구 봉천로49길 12 1층(봉천동)',
  'agent_comment': '',
  'agent_email': 'xzbmn11@gmail.com',
  'agent_lat': 37.48296543953569,
  'agent_lng': 126.95206597806002,
  'agent_local1': '서울시',
  'agent_local2': '관악구',
  'agent_mobile': '010-3733-9323',
  'agent_name': '수공인중개사사무소(임준혁)',
  'agent_no': 60776,
  'agent_phone': '02-875-0515',
  'bjd_code': '',
  'bonbun_code': '1667',
  'bubun_code': '21',
  'building': None,
  'building_type': '다세대건물',
  'contract': '서울특별시',
  'deposit': 6700,
  'description': '💚안녕하세요. 수부동산입니다.💚\r\n'
                 '\r\n'
                 ' \r\n'
                 '\r\n'
                 '💚100% 실매물 직접 형장확인 후 사진 촬영한 매물입니다.💚\r\n'
                 '\r\n'
                 ' \r\n'
                 '\r\n'
                 '💚가격대비 크기 좋은 방입니다.💚\r\n'
                 '\r\n'
                 ' \r\n'
                 '\r\n'
                 '\r\n'
                

In [13]:
filtered_result = export_items(result)

In [14]:
len(filtered_result)


26

In [15]:
def get_full_address(item):
    addr = item['address1']
    addr += ' '
    addr += item['address2'] or ''
    addr += ' '
    addr += item['address3'] or ''
    return addr

In [17]:
geo_based_dict = {}

In [18]:

for item in filtered_result:
    full_addr = get_full_address(item)
    item['address'] = full_addr
    if full_addr not in geo_based_dict:
        geo_based_dict[full_addr] = []
    geo_based_dict[full_addr].append(item)

In [19]:
first_key = next(iter(geo_based_dict))
pp.pprint(geo_based_dict[first_key][0])

{ '_floor': '5층',
  'address': '서울시 관악구 신림동 131-20 ',
  'address1': '서울시 관악구 신림동',
  'address2': '131-20',
  'address3': None,
  'agent_mobile': '010-2781-0314',
  'agent_name': '제이공인중개사사무소(장혜인)',
  'deposit': 6000,
  'description': '1.리모델링  원룸입니다!\r\n'
                 '2.없는것 없는 풀옵션!\r\n'
                 '3.주차 가능 합니다! (주차비 별도)\r\n'
                 '4.대로쪽에 위치한 원룸이라서 더욱 안전하고 좋습니다!\r\n'
                 '5.편의점,마트,먹거리,기타편의시설,버스정류장1분거리에 모두 있습니다!\r\n'
                 '6.서울대셔틀버스 정류장1분거리에 있습니다!\r\n'
                 '7.집주인분이 상주 하고 있어서 피드백이 빠릅니다!\r\n'
                 '*대출은 불가 합니다!\n'
                 '21.10.10에 직방에 등록된 매물입니다.',
  'id': 28940861,
  'manage_cost_inc': '수도,인터넷,TV',
  'room_type': '오픈형 원룸',
  'size': 6,
  'size_m2': 19.83,
  'status': '광고중',
  'title': '♥리모델링♥서울대셔틀1분♥프라임.합격의법학.1분♥',
  'updated_at2': '2021-10-10',
  'url': 'https://zigbang.com/home/oneroom/items/28940861'}


In [27]:
double_list = []
for addr in geo_based_dict.keys():
    double_list.append(geo_based_dict[addr][0])
double_list.sort(key=lambda x: x['size'], reverse=True)

In [29]:
with open('eggs.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = [
        'id',
        'title',
        'description',
        'deposit',
        'agent_name',
        'agent_mobile',
        '_floor',
        'address',
        'size',
        'size_m2',
        'status',
        'room_type',
        'manage_cost_inc',
        'updated_at2',
        'url',
    ]
    writer = csv.DictWriter(csvfile, extrasaction='ignore', fieldnames=fieldnames)
    writer.writeheader()
    for item in double_list:
        writer.writerow(item)